<a href="https://colab.research.google.com/github/nksubramanian/A/blob/main/ngrokworking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 7.3 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=dd944e2365092ff388754586085cfbe13dfcb0f586f4c50c04d4ad90388da642
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [2]:
from flask import Flask, render_template
import os
import threading

In [3]:
from flask import Flask, render_template
from pyngrok import ngrok
os.environ["FLASK_ENV"] = "development"

In [6]:
app = Flask(__name__)
port = 5000
ngrok.set_auth_token("1yXn3NSEMkqwyXHoBjWGC7sw1ul_754qikbMjYAP1xC68G4sT")
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel\"{}\"-> \"http://127.0.0.1:{}\"".format(public_url,port))
app.config["BASE_URL"] = public_url
@app.route("/")
def hello():
    return "Hello"
threading.Thread(target=app.run, kwargs={"use_reloader":False}).start()

PyngrokNgrokHTTPError: ignored